<a href="https://colab.research.google.com/github/Pedro2um/GameA/blob/main/instascrapingTEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas necessárias

In [ ]:
!pip install instagrapi

# Credenciais: login e senha

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open("drive/MyDrive/credentials.txt", "r") as f:
  ACCOUNT_USERNAME, ACCOUNT_PASSWORD = f.read().splitlines()

In [ ]:
print(ACCOUNT_USERNAME, ACCOUNT_PASSWORD)

carlinhos00027 01011901


# Fazer login e salvar sessão, segue abaixo os motivos:


*   Simular um usuário real
*   Ninguém em sã consciência sai do dispositivo e depois loga novamente



In [ ]:
from instagrapi import Client
cl = Client()
cl.delay_range = [2, 4] # setting delay, simulating real user
cl.login(ACCOUNT_USERNAME, ACCOUNT_PASSWORD, verification_code=957631)

True

In [ ]:
cl.dump_settings("drive/MyDrive/" + ACCOUNT_USERNAME + "_" + "session.json")

True

In [ ]:
login = "drive/MyDrive/" + ACCOUNT_USERNAME + "_" + "session.json"

In [ ]:
%cat $login

{
    "uuids": {
        "phone_id": "9da64b4f-8bf2-45ab-b6ee-29f24a641de7",
        "uuid": "7d6f2410-5f0d-46ea-88ce-b150b942fa20",
        "client_session_id": "6635426c-528b-4308-84e8-a8be56453880",
        "advertising_id": "aadeae5e-226c-48ad-9846-17df8ac4b2db",
        "android_device_id": "android-95888d94531d95b9",
        "request_id": "e583ec46-2a2b-43fe-9da3-ab42dbd7ca9f",
        "tray_session_id": "80e62ccd-a598-41e7-9bd3-fdfa96909235"
    },
    "mid": "Zh8CzQABAAGPu47FC_R8_PZ12b19",
    "ig_u_rur": null,
    "ig_www_claim": null,
    "authorization_data": {
        "ds_user_id": "66078071525",
        "sessionid": "66078071525%3AOKsEVaWym0cFP1%3A9%3AAYcyHc5snswjb5UN8MdgH4djDUeXDprzXGbIuAow5A"
    },
    "cookies": {},
    "last_login": 1713308384.6398182,
    "device_settings": {
        "app_version": "269.0.0.18.75",
        "android_version": 26,
        "android_release": "8.0.0",
        "dpi": "480dpi",
        "resolution": "1080x1920",
        "manufacturer": "On

# Ler sessão e utilizar usuário

In [ ]:
# reference:
# https://subzeroid.github.io/instagrapi/usage-guide/best-practices.html
from instagrapi import Client
from instagrapi.exceptions import LoginRequired
import logging

logger = logging.getLogger()

def login_user():
    """
    Attempts to login to Instagram using either the provided session information
    or the provided username and password.
    """

    cl = Client()
    session = cl.load_settings(login)

    login_via_session = False
    login_via_pw = False

    if session:
        try:
            cl.set_settings(session)
            cl.login(ACCOUNT_USERNAME, ACCOUNT_PASSWORD)

            # check if session is valid
            try:
                cl.get_timeline_feed()
            except LoginRequired:
                logger.info("Session is invalid, need to login via username and password")

                old_session = cl.get_settings()

                # use the same device uuids across logins
                cl.set_settings({})
                cl.set_uuids(old_session["uuids"])

                cl.login(ACCOUNT_USERNAME, ACCOUNT_PASSWORD)
            login_via_session = True
        except Exception as e:
            logger.info("Couldn't login user using session information: %s" % e)

    if not login_via_session:
        try:
            logger.info("Attempting to login via username and password. username: %s" % ACCOUNT_USERNAME)
            if cl.login(ACCOUNT_USERNAME, ACCOUNT_PASSWORD):
                login_via_pw = True
        except Exception as e:
            logger.info("Couldn't login user using username and password: %s" % e)

    if not login_via_pw and not login_via_session:
        raise Exception("Couldn't login user with either password or session")

# Teste com hashtag "Festa da Penha"

In [ ]:
hashtag = "festadapenha"

In [ ]:
from random import randint
from time import sleep

# Classe útil, Account

In [ ]:
from instagrapi import Client
from instagrapi.exceptions import (
    BadPassword,
    ChallengeRequired,
    FeedbackRequired,
    LoginRequired,
    PleaseWaitFewMinutes,
    RecaptchaChallengeForm,
    ReloginAttemptExceeded,
    SelectContactPointRecoveryForm,
)


class Account:
    username = ""
    password = ""

    def get_client(self):
        """We return the client class, in which we automatically handle exceptions
        You can move the "handle_exception" above or into an external module
        """

        def handle_exception(client, e):
            if isinstance(e, BadPassword):
                client.logger.exception(e)
                client.set_proxy(self.next_proxy().href)
                if client.relogin_attempt > 0:
                    self.freeze(str(e), days=7)
                    raise ReloginAttemptExceeded(e)
                client.settings = self.rebuild_client_settings()
                return self.update_client_settings(client.get_settings())
            elif isinstance(e, LoginRequired):
                client.logger.exception(e)
                client.relogin()
                return self.update_client_settings(client.get_settings())
            elif isinstance(e, ChallengeRequired):
                api_path = client.last_json.get("challenge", {}).get("api_path")
                if api_path == "/challenge/":
                    client.set_proxy(self.next_proxy().href)
                    client.settings = self.rebuild_client_settings()
                else:
                    try:
                        client.challenge_resolve(client.last_json)
                    except ChallengeRequired as e:
                        self.freeze("Manual Challenge Required", days=2)
                        raise e
                    except (
                        ChallengeRequired,
                        SelectContactPointRecoveryForm,
                        RecaptchaChallengeForm,
                    ) as e:
                        self.freeze(str(e), days=4)
                        raise e
                    self.update_client_settings(client.get_settings())
                return True
            elif isinstance(e, FeedbackRequired):
                message = client.last_json["feedback_message"]
                if "This action was blocked. Please try again later" in message:
                    self.freeze(message, hours=12)
                    # client.settings = self.rebuild_client_settings()
                    # return self.update_client_settings(client.get_settings())
                elif "We restrict certain activity to protect our community" in message:
                    # 6 hours is not enough
                    self.freeze(message, hours=12)
                elif "Your account has been temporarily blocked" in message:
                    """
                    Based on previous use of this feature, your account has been temporarily
                    blocked from taking this action.
                    This block will expire some day.
                    """
                    self.freeze(message)
            elif isinstance(e, PleaseWaitFewMinutes):
                self.freeze(str(e), hours=1)
            raise e

        cl = Client()
        cl.handle_exception = handle_exception
        cl.login(self.username, self.password)
        return cl

# Testando limites

In [ ]:
def test(t, lim):
  login_user()
  print(t)
  if t > 100:
    print("FUCK!")
    return -1
  try:
    medias = cl.hashtag_medias_recent(hashtag, lim)
    return medias
  except:
    login_user()
    test(t+1, lim)


# Limites e tempos

OBS: **Não estou usando proxy, pode ser o motivo do problema, tentarei amanhã novamente 17/04/2024**
*   1000 posts = 3min
*   10000 posts não é possivel com a abordagem atual



In [ ]:
lim = 1000

In [ ]:
medias = test(0, lim)
if type(medias) == int or type(medias) == None:
  print("LIMIT IS", lim)
else:
  M = [ dict(m) for m in medias ] # visualizar em dataframe

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [ ]:
import pandas as pd

In [ ]:
df_medias = pd.DataFrame(M)

In [ ]:
df_medias.to_csv('drive/MyDrive/df_medias' + "lim=" + str(lim) + ".csv")

# Dataset 1000 posts


1.   algumas funções úteis abaixo vão auxiliar (possívelmente) nas querys
2.   muitas colunas obtidas a partir de uma hashtag. Pensando se é possível apenas buscar com termos utilizando a API...



In [ ]:
import pandas as pd

In [ ]:
dataframe = pd.read_csv('drive/MyDrive/df_medias' + "lim=1000" + ".csv")

In [ ]:
dataframe.columns.tolist()

['Unnamed: 0',
 'pk',
 'id',
 'code',
 'taken_at',
 'media_type',
 'image_versions2',
 'product_type',
 'thumbnail_url',
 'location',
 'user',
 'comment_count',
 'comments_disabled',
 'commenting_disabled_for_viewer',
 'like_count',
 'play_count',
 'has_liked',
 'caption_text',
 'accessibility_caption',
 'usertags',
 'sponsor_tags',
 'video_url',
 'view_count',
 'video_duration',
 'title',
 'resources',
 'clips_metadata']

In [ ]:
dataframe

,Unnamed: 0,pk,id,code,taken_at,media_type,image_versions2,product_type,thumbnail_url,location,...,caption_text,accessibility_caption,usertags,sponsor_tags,video_url,view_count,video_duration,title,resources,clips_metadata
0,0,3235801736517688967,3235801736517688967_57855250491,Czn4IkMup6H,2023-11-14 10:27:27+00:00,8,"{'candidates': [{'width': 1080, 'height': 1350...",carousel_container,NaN,pk=497367445 name='Espírito Santo' phone='' we...,...,🎬🎤 Um projeto incrível está sendo produzido pa...,NaN,"[Usertag(user=UserShort(pk='1388454361', usern...",[],NaN,0,0.000,NaN,"[Resource(pk='3235801729932537410', video_url=...",{}
1,1,3339091257773224253,3339091257773224253_57855250491,C5W1c7EL3E9,2024-04-04 22:46:09+00:00,2,"{'candidates': [{'width': 360, 'height': 640, ...",clips,https://scontent-lax3-2.cdninstagram.com/v/t51...,pk=614132318606195 name='Convento da Penha' ph...,...,"📿 Confira o resumo dessa quarta-feira (03), na...",NaN,"[Usertag(user=UserShort(pk='2135317063', usern...",[],https://scontent-lax3-2.cdninstagram.com/o1/v/...,0,88.933,NaN,[],{'achievements_info': {'show_achievements': Fa...
2,2,3081447350954851899,3081447350954851899_57855250491,CrDgAobu347,2023-04-15 11:12:51+00:00,1,"{'candidates': [{'width': 819, 'height': 819, ...",feed,https://scontent-lax3-2.cdninstagram.com/v/t51...,pk=900994 name='Catedral Metropolitana de Vitó...,...,🙏 Hoje acontece a maior Romaria da Festa da Pe...,NaN,"[Usertag(user=UserShort(pk='2135317063', usern...",[],NaN,0,0.000,NaN,[],{}
3,3,3335344704231968005,3335344704231968005_202734136,C5JhlZLOg0F,2024-03-30 18:41:34+00:00,8,"{'candidates': [{'width': 1080, 'height': 1346...",carousel_container,NaN,pk=497367445 name='Espírito Santo' phone='' we...,...,Em homenagem às celebrações da #FestadaPenha 2...,NaN,[],[],NaN,0,0.000,NaN,"[Resource(pk='3335344694257892502', video_url=...",{}
4,4,3338956952535091359,3338956952535091359_57855250491,C5WW6hkLMif,2024-04-04 18:19:52+00:00,2,"{'candidates': [{'width': 360, 'height': 640, ...",clips,https://scontent-lax3-1.cdninstagram.com/v/t51...,pk=614132318606195 name='Convento da Penha' ph...,...,📸 A Festa da Penha preparou ambientes incrivei...,NaN,"[Usertag(user=UserShort(pk='2135317063', usern...",[],https://scontent-lax3-2.cdninstagram.com/o1/v/...,0,35.900,NaN,[],{'achievements_info': {'show_achievements': Fa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,2026911723782801792,2026911723782801792_2135317063,BwhCXlHJgmA,2019-04-21 11:38:48+00:00,8,"{'candidates': [{'width': 1080, 'height': 1350...",carousel_container,NaN,pk=272080699922528 name='Cobilandia' phone='' ...,...,Paz e Bem! Em instantes terá início a Romaria ...,NaN,[],[],NaN,0,0.000,NaN,"[Resource(pk='2026911719554900825', video_url=...",{}
996,996,3341082622673378269,3341082622673378269_44400894608,C5d6PGJrVvd,2024-04-07 16:41:47+00:00,1,"{'candidates': [{'width': 613, 'height': 766, ...",feed,https://scontent-lax3-1.cdninstagram.com/v/t51...,NaN,...,"Jesus, eu confio em vos!\n\n#jesusmisericordio...",NaN,"[Usertag(user=UserShort(pk='12414837348', user...",[],NaN,0,0.000,NaN,[],{}
997,997,1752186418735964431,1752186418735964431_1331591535,BhRBEOtBWEP,2018-04-07 10:40:53+00:00,2,"{'candidates': [{'width': 612, 'height': 612, ...",feed,https://scontent-lax3-1.cdninstagram.com/v/t51...,pk=614132318606195 name='Convento da Penha' ph...,...,Ainda estamos tentando encontrar palavras para...,NaN,[],[],https://scontent-lax3-1.cdninstagram.com/o1/v/...,0,60.000,NaN,[],{}
998,998,3342768151232091089,3342768151232091089_199024576,C5j5ew8rZPR,2024-04-10 00:30:38+00:00,8,"{'candidates': [{'width': 1080, 'height': 1350...",carousel_container,NaN,pk=256405639 name='Villa Velha Es' phone='' we...,...,Romaria dos Ciclistas 2024\n@jesse.junior.56\n...,NaN,"[Usertag(user=UserShort(pk='5663607209', usern...",[],NaN,0,0.000,NaN,"[Resource(pk='3342768138858874335', video_url=...",{}


In [ ]:
dataframe[['caption_text']]

,caption_text
0,🎬🎤 Um projeto incrível está sendo produzido pa...
1,"📿 Confira o resumo dessa quarta-feira (03), na..."
2,🙏 Hoje acontece a maior Romaria da Festa da Pe...
3,Em homenagem às celebrações da #FestadaPenha 2...
4,📸 A Festa da Penha preparou ambientes incrivei...
...,...
995,Paz e Bem! Em instantes terá início a Romaria ...
996,"Jesus, eu confio em vos!\n\n#jesusmisericordio..."
997,Ainda estamos tentando encontrar palavras para...
998,Romaria dos Ciclistas 2024\n@jesse.junior.56\n...


In [ ]:
def filter_by_terms(list_of_hashtags, df, target_column): # a hashtag is just a term with a '#' as a prefix
  mask = df[target_column].str.contains('|'.join(list_of_hashtags), case=False)
  return df[mask]

In [ ]:
df_sub = filter_by_terms(['#romaria', '#conventodapenha'], dataframe, 'caption_text')

#Query equivalente no CrawdTangle


*   **#festadapenha** AND (#romaria OR #conventodapenha)
*   Fazer o Parser da query talvez seja problemático, talvez não






# Ideia de Parser


*   transformar formula em CNF ou DNF
*   assumir formula possui <= 1000 operadores booleanos, ou seja, um número constante

